In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter 
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate
from langchain import OpenAI, ConversationChain

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import PyPDF2
import PyPDF2
import numpy as np 

In [2]:
import os 
# Load openai api key 
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Load Pinecone API key 
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Load pinecode api env 
pinecone_api_env = os.environ.get('PINECONE_API_ENV')

In [3]:
# Load openai api key 
openai_api_key = os.environ.get('OPENAI_API_KEY')
# Load Pinecone API key 
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
# Load pinecode api env 
pinecone_api_env = os.environ.get('PINECONE_API_ENV')

# Extract text from the PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ''
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extractText()
    return text

def get_all_pdf_files_in_directory(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]

In [4]:
import pinecone

# Initialize a Pinecone client

pinecone.init(api_key=pinecone_api_key, api_env=pinecone_api_env)
#help(pinecone.init)

c:\Users\Leonce Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
# Pick a name for the new index
index_name = 'semantic-text-search'

In [6]:
import pandas as pd
import numpy as np
import time
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import sqlite3

pd.set_option('display.max_colwidth', 200)

In [7]:
import os
import PyPDF2
import pinecone
from sentence_transformers import SentenceTransformer

In [8]:
# Generate vector embeddings
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [9]:
data_directory = "data/"
pdf_files = get_all_pdf_files_in_directory(data_directory)

In [10]:
bnr_report_reader = UnstructuredPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
bnr_report_reader_data = bnr_report_reader.load()

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
# conversation = ConversationChain(llm=llm, verbose=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pdf2image\pdf2image.py: │
│ 568 in pdfinfo_from_path                                                                         │
│                                                                                                  │
│   565 │   │   env = os.environ.copy()                                                            │
│   566 │   │   if poppler_path is not None:                                                       │
│   567 │   │   │   env["LD_LIBRARY_PATH"] = poppler_path + ":" + env.get("LD_LIBRARY_PATH", "")   │
│ ❱ 568 │   │   proc = Popen(command, env=env, stdout=PIPE, stderr=PIPE)                           │
│   569 │   │                                                                                      │
│   570 │   │   try:                                                                               │
│   571 │   │   │   out, err = proc.communicate(timeout=timeout)                                   │
│                                                                                                  │
│ C:\Users\Leonce Nshuti\miniconda3\lib\subprocess.py:858 in __init__                              │
│                                                                                                  │
│    855 │   │   │   │   │   self.stderr = io.TextIOWrapper(self.stderr,                           │
│    856 │   │   │   │   │   │   │   encoding=encoding, errors=errors)                             │
│    857 │   │   │                                                                                 │
│ ❱  858 │   │   │   self._execute_child(args, executable, preexec_fn, close_fds,                  │
│    859 │   │   │   │   │   │   │   │   pass_fds, cwd, env,                                       │
│    860 │   │   │   │   │   │   │   │   startupinfo, creationflags, shell,                        │
│    861 │   │   │   │   │   │   │   │   p2cread, p2cwrite,                                        │
│                                                                                                  │
│ C:\Users\Leonce Nshuti\miniconda3\lib\subprocess.py:1311 in _execute_child                       │
│                                                                                                  │
│   1308 │   │   │                                                                                 │
│   1309 │   │   │   # Start the process                                                           │
│   1310 │   │   │   try:                                                                          │
│ ❱ 1311 │   │   │   │   hp, ht, pid, tid = _winapi.CreateProcess(executable, args,                │
│   1312 │   │   │   │   │   │   │   │   │   │    # no special security                            │
│   1313 │   │   │   │   │   │   │   │   │   │    None, None,                                      │
│   1314 │   │   │   │   │   │   │   │   │   │    int(not close_fds),                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [WinError 2] The system cannot find the file specified

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\1713438424.py:2 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\1713438424.py'                                    │
│                 

In [11]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
pages = loader.load_and_split()

In [ ]:
# Pick a name for the new index
simple_index_name = 'stocks-trends'

In [12]:
# Check whether the index with the same name already exists
if simple_index_name in pinecone.list_indexes():
    pinecone.delete_index(simple_index_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\349161899.py:2 in <module>             │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\349161899.py'                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'simple_index_name' is not defined

In [13]:
# Create a new index
pinecone.create_index(name=simple_index_name, dimension=128)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\2531901648.py:2 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\2531901648.py'                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'simple_index_name' is not defined

In [14]:
# Define nlp
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_named_entities(text_batch):
    # extract named entities using the NER pipeline
    extracted_batch = nlp(text_batch)
    entities = []
    # loop through the results and only select the entity names
    for text in extracted_batch:
        ne = [entity["word"] for entity in text]
        entities.append(ne)
    return entities

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\2630759853.py:3 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\2630759853.py'                                    │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\spacy\__init__.py:54 in │
│ load                                                                                             │
│                                                                                                  │
│   51 │   │   keyed by section values in dot notation.                                            │
│   52 │   RETURNS (Language): The loaded nlp object.                                              │
│   53 │   """                                                                                     │
│ ❱ 54 │   return util.load_model(                                                                 │
│   55 │   │   name,                                                                               │
│   56 │   │   vocab=vocab,                                                                        │
│   57 │   │   disable=disable,                                                                    │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\spacy\util.py:449 in    │
│ load_model                                                                                       │
│                                                                                                  │
│    446 │   │   return load_model_from_path(name, **kwargs)  # type: ignore[arg-type]             │
│    447 │   if name in OLD_MODEL_SHORTCUTS:                                                       │
│    448 │   │   raise IOError(Errors.E941.format(name=name, full=OLD_MODEL_SHORTCUTS[name]))  #   │
│ ❱  449 │   raise IOError(Errors.E050.format(name=name))                                          │
│    450                                                                                           │
│    451                                                                                           │
│    452 def load_model_from_package(                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data
directory.

In [15]:
pinecone.list_indexes()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\2524688261.py:1 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\2524688261.py'                                    │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\manage.py:185  │
│ in list_indexes                                                                                  │
│                                                                                                  │
│   182 def list_indexes():                                                                        │
│   183 │   """Lists all indexes."""                                                               │
│   184 │   api_instance = _get_api_instance()                                                     │
│ ❱ 185 │   response = api_instance.list_indexes()                                                 │
│   186 │   return response                                                                        │
│   187                                                                                            │
│   188                                                                                            │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\core\client\ap │
│ i_client.py:776 in __call__                                                                      │
│                                                                                                  │
│   773 │   │   api_instance.configure_index.callable or self.callable in this class               │
│   774 │   │                                                                                      │
│   775 │   │   """                                                                                │
│ ❱ 776 │   │   return self.callable(self, *args, **kwargs)                                        │
│   777 │                                                                                          │
│   778 │   def call_with_http_info(self, **kwargs):                                               │
│   779                                                                                            │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\core\client\ap │
│ i\index_operations_api.py:1132 in __list_indexes                                                 │
│                                                                                                  │
│   1129 │   │   │   │   '_check_return_type', True                                                │
│   1130 │   │   │   )                                                                             │
│   1131 │   │   │   kwargs['_host_index'] = kwargs.get('_host_index')                             │
│ ❱ 1132 │   │   │   return self.call_with_http_info(**kwargs)                                     │
│   1133 │   │                                                                                     │
│   1134 │   │   self.list_indexes = _Endpoint(              

In [ ]:
pinecone.describe_index("pinecone-index")

In [ ]:
pinecone.create_index("example-index", dimension=128, metric="euclidean", pods=4, pod_type="s1.x1")

In [ ]:
pinecone.create_index("example-index", dimension=128, source_collection="example-collection")

In [ ]:
pinecone.configure_index("my_index", pod_type="s1.x2")

In [ ]:
pinecone.describe_index("example-index")

In [ ]:
pinecone.configure_index("example-index", replicas=4)

In [16]:
from pprint import pprint
# Define retriever
retriever = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Define index
index = pinecone.Index()

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["title"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\2707508682.py:6 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\2707508682.py'                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: __init__() missing 1 required positional argument: 'index_name'

In [17]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

146: NBR Annual Report | 2021 - 2022147
 # Policy Paper Objective Findings Recommendations
1Monitoring the 
Economic Impact 
of COVID-19To describe how various 
high-frequency indicators, 
as well as the computation 
of weekly index of economic 
activity, were used in Rwanda 
to monitor the economic 
impact of COVID-19. The pandemic has substantially 
weakened economic performance 
through demand and supply shocks 
and affected all sectors, especially 
the manufacturing and service 
sectors.Proposed various measures to 
recover back to the pre-pandemic 
level successfully.
2Monetary policy, 
credit growth, and 
Economic activity 
in RwandaTo provide a deeper analysis 
of the effect of monetary 
policy on target variables, 
such as credit, output, and 
inflation in Rwanda(1) A positive shock on the 
interbank reduces the credit by one 
percentage point, and positive credit 
shocks allow agents to spend more, 
thus a short but positive reaction for 
the Gross Domestic Product (GDP). 
(2)

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)
print (f'Now you have {len(texts)} documents')

Now you have 494 documents


In [19]:
import pinecone

pinecone.init(api_key=pinecone_api_key)

pinecone.create_index("national_bank_index", dimension=1024)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\2769993483.py:5 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\2769993483.py'                                    │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\manage.py:118  │
│ in create_index                                                                                  │
│                                                                                                  │
│   115 │   """                                                                                    │
│   116 │   api_instance = _get_api_instance()                                                     │
│   117 │                                                                                          │
│ ❱ 118 │   api_instance.create_index(create_request=CreateRequest(                                │
│   119 │   │   name=name,                                                                         │
│   120 │   │   dimension=dimension,                                                               │
│   121 │   │   index_type=index_type,                                                             │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\core\client\ap │
│ i_client.py:776 in __call__                                                                      │
│                                                                                                  │
│   773 │   │   api_instance.configure_index.callable or self.callable in this class               │
│   774 │   │                                                                                      │
│   775 │   │   """                                                                                │
│ ❱ 776 │   │   return self.callable(self, *args, **kwargs)                                        │
│   777 │                                                                                          │
│   778 │   def call_with_http_info(self, **kwargs):                                               │
│   779                                                                                            │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pinecone\core\client\ap │
│ i\index_operations_api.py:370 in __create_index                                                  │
│                                                                                                  │
│    367 │   │   │   │   '_check_return_type', True                                                │
│    368 │   │   │   )                                                                             │
│    369 │   │   │   kwargs['_host_index'] = kwargs.get('_host_index')                             │
│ ❱  370 │   │   │   return self.call_with_http_info(**kwargs)                                     │
│    371 │   │                                                                                     │
│    372 │   │   self.create_index = _Endpoint(              

In [ ]:
pinecone.init(api_key="YOUR_API_KEY", environment="YOUR_ENVIRONMENT")
index = pinecone.Index("example-index")

index_stats_response = index.describe_index_stats()

In [20]:
import chromadb
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# create a new index 
index = client.create_index("bnr_index")

# add documents to the index
for text in texts:
    index.add_document(text)

# search for similar documents  
results = index.search("What are the top economic challenges facing Rwanda?", k=2)
for result in results:
    print(result)

Using embedded DuckDB without persistence: data will be transient


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_24696\3976387204.py:6 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_24696\\3976387204.py'                                    │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\chromadb\api\local.py:3 │
│ 06 in create_index                                                                               │
│                                                                                                  │
│   303 │   │   return self._db.raw_sql(raw_sql)                                                   │
│   304 │                                                                                          │
│   305 │   def create_index(self, collection_name: str):                                          │
│ ❱ 306 │   │   collection_uuid = self._db.get_collection_uuid_from_name(collection_name)          │
│   307 │   │   self._db.create_index(collection_uuid=collection_uuid)                             │
│   308 │   │   return True                                                                        │
│   309                                                                                            │
│                                                                                                  │
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\chromadb\db\duckdb.py:7 │
│ 1 in get_collection_uuid_from_name                                                               │
│                                                                                                  │
│    68 │   #  UTILITY METHODS                                                                     │
│    69 │   #                                                                                      │
│    70 │   def get_collection_uuid_from_name(self, name):                                         │
│ ❱  71 │   │   return self._conn.execute("SELECT uuid FROM collections WHERE name = ?", [name])   │
│    72 │   │   │   0                                                                              │
│    73 │   │   ][0]                                                                               │
│    74                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
import pinecone

pinecone.init(api_key=pinecone_api_key)
index = pinecone.Index("example-index")

In [ ]:
metadata_config = {'indexed': ['color']}

In [ ]:
# Use Pinecone to implement a vector store 
pinecone_index = Pinecone(index_name="example-index", metadata_config=metadata_config)

# Use OpenAI to implement an embedding model
openai_embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Use Weaviate to implement a vector store  

weaviate_index = Weaviate("http://localhost:8080", "bnr_index")

# Use FAISS to implement a vector store
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))

In [21]:

# Use approximate nearest neighbor search to find similar documents
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

146: NBR Annual Report | 2021 - 2022147
 # Policy Paper Objective Findings Recommendations
1Monitoring the 
Economic Impact 
of COVID-19To describe how various 
high-frequency indicators, 
as well as the computation 
of weekly index of economic 
activity, were used in Rwanda 
to monitor the economic 
impact of COVID-19. The pandemic has substantially 
weakened economic performance 
through demand and supply shocks 
and affected all sectors, especially 
the manufacturing and service 
sectors.Proposed various measures to 
recover back to the pre-pandemic 
level successfully.
2Monetary policy, 
credit growth, and 
Economic activity 
in RwandaTo provide a deeper analysis 
of the effect of monetary 
policy on target variables, 
such as credit, output, and 
inflation in Rwanda(1) A positive shock on the 
interbank reduces the credit by one 
percentage point, and positive credit 
shocks allow agents to spend more, 
thus a short but positive reaction for 
the Gross Domestic Product (GDP). 
(2)